# Teste de Sistema de Detecção de Fraudes

Teste utilizando o OneClassSVM, que é uma implementação de Support Vector Machine para Unsupervised Learning

Nessa versão o teste é bem simples, utiliza só duas features e não faz tantos testes de configuração da SVM. Quero fazer uma outra versão pra comparar SVM, Logistic Regression e Neural Network. Al[em disso, usar GridSearch para teste de parâmetros etc

**Segunda Versão** Vou tentar usar o GridSearch para definir parâmetros


In [ ]:
from sklearn import svm
import numpy as np
from sklearn import preprocessing





- Features layout: cod.cidade, valor compra
  - cod. cidade = 1 sp, 2 jundiai, 3 campinas, 4 sorocaba, 5 internet
  - local: 1 shoppping, 2 posto gasolina, 3 feira livre, 4 restaurante


** Create a list for training using a random number generator **



In [ ]:
import random

place  = np.random.randint(1,4,150)
price  = np.random.uniform(50.0,200,150)

X = np.column_stack((place,price))

print X.shape
print X[:,0].shape
print X


test_place  = np.random.randint(1,8,100)
test_price  = np.random.uniform(50.0,200,100)

X_test = np.ones((100,2))

X_test[:,0] = test_place.reshape(100,)
X_test[:,1] = test_price.reshape(100,)

#print X_test


### ** Normalize data for SVM training **

This is a ** very ** important step, specially because the features are in a very different range

** Nota: ** o Objeto standard, criado aqui, será utilizado abaixo, nos testes, para normalizar os dados de testes baseados nos parametros de normalização usados aqui no treinamento

In [ ]:

standard = preprocessing.StandardScaler().fit(X)
df_std = standard.transform(X)
X = df_std

print type(df_std)
print df_std.shape
#print X


df_std_test = standard.transform(X_test)
X_test = df_std_test
#print X_test

## Usando o GridSearchCV para a escolha dos Parâmetros gamma e nu

Para escolher o resultado, olhar o gráfico e os indices impressos abaixo. Note que a cada vez que rodamos o script, os dados são gerados novamente (random) por isso não dá pra usar sempre o melhor valor que aparece aqui. Se os valores fossem fixos, ou reais, seria possível fazer o uso.

In [ ]:

from sklearn.grid_search import GridSearchCV

gamma_range = np.linspace (0.1,0.9,9)
nu_range   = np.linspace (0.1,0.9,9)
param_grid = dict (gamma=gamma_range,nu=nu_range)

#print param_grid
                   
clsf = svm.OneClassSVM(nu=0.1,kernel='rbf',gamma=0.1)


grid = GridSearchCV(clsf,param_grid,cv=10,scoring='accuracy')


y = np.ones((150,1))
#print y
grid.fit(X,y)

#print "decision function"
#clsf.decision_function(X)

for index,res in enumerate(grid.grid_scores_):
    print "idnex: %d  gamma: %f  nu: %f  mean: %f" % (index,res.parameters['gamma'],res.parameters['nu'],res.mean_validation_score)

grid_mean_scores = [result.mean_validation_score for result in grid.grid_scores_]

import matplotlib.pyplot as plt
%matplotlib inline


plt.plot (grid_mean_scores)
plt.xlabel('index of gridsearch teste')
plt.ylabel('grid mean score')
plt.show()




In [ ]:
# usando valores padrões de gamma e nu
clsf = svm.OneClassSVM(nu=0.1,kernel='rbf',gamma=0.1)
clsf.fit(X)

output = clsf.predict (X)


print "Corretos: %d " % len(output[output == 1])
print "Fraude: %d " % len(output[output == -1])




## Usando o X_test para fazer o predict

agora, o X_test contém uma quantidade grande de valores que deveriam ser considerados FRAUDE

In [ ]:
output = clsf.predict (X_test)


print "Corretos: %d " % len(output[output == 1])
print "Fraude: %d " % len(output[output == -1])

